In [1]:
import re
import os
import json
import json_repair
import pandas as pd
from openai import OpenAI
from pydantic import BaseModel
from dotenv import load_dotenv
from pqdm.processes import pqdm
from datasets import load_dataset

# 환경변수 로드
load_dotenv("./credit-env")

# 이시간에는 합성데이터를 만들기 위해 기존의 데이터가 어떻게 구성되었는지 확인 
sql_ds = load_dataset("won75/text_to_sql_ko")
sql_ds2 = load_dataset("Clinton/Text-to-sql-v1")
synthetic_text_to_sql = load_dataset("gretelai/synthetic_text_to_sql")

/home/ubuntu/anaconda3/envs/ds_ggls/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sql_ds["train"][0]

{'TEXT': '가장 많이 팔린 상위 5개 상품의 이름과 판매 수량을 알고 싶습니다.\n',
 'MySQL': 'SELECT\n    p.product_name,\n    SUM(oi.quantity) as total_sold\nFROM\n    sales_db.order_items oi\nJOIN\n    sales_db.products p ON oi.product_id = p.product_id\nGROUP BY\n    p.product_name\nORDER BY\n    total_sold DESC\nLIMIT 5;',
 'Schema': 'DB: sales_db\nTABLE DDL:\nCREATE TABLE `order_items` (\n    order_item_id BIGINT NOT NULL,\n    order_id BIGINT,\n    product_id BIGINT,\n    quantity INTEGER,\n    unit_price DECIMAL(10, 2),\n    total_price DECIMAL(10, 2),\n    item_status VARCHAR(50),\n    created_at DATETIME,\n    PRIMARY KEY (order_item_id),\n    CONSTRAINT `order_items_ibfk_1` FOREIGN KEY(product_id) REFERENCES `products` (product_id)\n)\n\nDB: sales_db\nTABLE DDL:\nCREATE TABLE `products` (\n    product_id BIGINT NOT NULL,\n    product_name VARCHAR(200),\n    category VARCHAR(100),\n    original_price DECIMAL(10, 2),\n    product_type VARCHAR(50),\n    is_active TINYINT(1),\n    created_at DATETIME,\n    PR

In [3]:
sql_ds2["train"][0]

{'instruction': 'Name the home team for carlton away team',
 'input': 'CREATE TABLE table_name_77 (\n    home_team VARCHAR,\n    away_team VARCHAR\n)',
 'response': 'SELECT home_team FROM table_name_77 WHERE away_team = "carlton"',
 'source': 'sql_create_context',
 'text': 'Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables. ### Instruction: Name the home team for carlton away team ### Input: CREATE TABLE table_name_77 (\n    home_team VARCHAR,\n    away_team VARCHAR\n) ### Response: SELECT home_team FROM table_name_77 WHERE away_team = "carlton"'}

In [4]:
synthetic_text_to_sql["train"][0]

{'id': 5097,
 'domain': 'forestry',
 'domain_description': 'Comprehensive data on sustainable forest management, timber production, wildlife habitat, and carbon sequestration in forestry.',
 'sql_complexity': 'single join',
 'sql_complexity_description': 'only one join (specify inner, outer, cross)',
 'sql_task_type': 'analytics and reporting',
 'sql_task_type_description': 'generating reports, dashboards, and analytical insights',
 'sql_prompt': 'What is the total volume of timber sold by each salesperson, sorted by salesperson?',
 'sql_context': "CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE); INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES (1, 1, 120, '2021-01-01'), (2, 1, 150, '2021-02-01'), (3, 2, 180, '2021-01-01');",
 'sql'

In [5]:
category_metadata = {
    # 분야(Domain)를 한글로 세부화
    "Domain": [
        "금융",
        "의료",
        "공공 보건",
        "전자상거래",
        "교육",
        "교통",
        "부동산",
        "엔터테인먼트",
        "스포츠",
        "소셜 미디어",
        "숙박 및 관광",
        "정부 및 행정",
        "에너지",
        "농업",
        "제조업",
        "통신",
        "유통 및 물류",
        "환경",
        "보험",
        "법률",
        "인적 자원 관리(HR)"
    ],

    # SQL 작업 유형(SQL Task Type)을 한글로 세분화 및 확장
    "SQL Task Type": [
        "분석 및 리포트 생성",
        "데이터 추가 및 입력",
        "데이터 수정 및 갱신",
        "데이터 삭제",
        "기본 데이터 조회",
        "데이터베이스 객체 생성 및 관리",
        "사용자 관리 및 권한 설정",
        "감사(audit) 및 로그 기록",
        "데이터 품질 관리 및 검증",
        "성능 최적화 및 관리",
        "백업 및 복구 관리"
    ],

    # SQL Complexity (쿼리 복잡성)
    "SQL Complexity": [
        "basic SQL",         # 간단한 SELECT, INSERT, UPDATE, DELETE 등 기본적인 SQL 문장
        "aggregation",       # COUNT, SUM, AVG, MIN, MAX 등의 집계함수 및 GROUP BY, HAVING 포함
        "single join",       # 하나의 JOIN만 포함된 쿼리 (두 테이블 간의 간단한 JOIN)
        "multiple joins",    # 두 개 이상의 JOIN을 포함한 복잡한 쿼리
        "subqueries",        # 서브쿼리(하위 쿼리)가 포함된 SQL 문장
        "window functions",  # ROW_NUMBER(), RANK(), DENSE_RANK()와 같은 윈도우 함수 포함
        "set operations",    # UNION, INTERSECT, EXCEPT와 같은 집합 연산자 포함
        "CTEs"               # 공통 테이블 표현식(Common Table Expressions)을 사용한 WITH 절 포함 쿼리
    ],


    # 쿼리 타입
    "Type": [
        "SELECT",
        "UPDATE",
        "INSERT",
        "DELETE"
    ],
}


In [6]:
import random

def select_realistic_main_syntax(difficulty=3):
    """
    난이도에 따라 현실적인 SQL 문법 조합을 생성하는 함수
    
    Args:
        difficulty (int): 1-5 사이의 난이도 값
        
    Returns:
        list: 선택된 SQL 문법 리스트
    """
    # SQL 문법을 카테고리별로 분류하여 딕셔너리로 저장함. 각 카테고리는 관련된 SQL 문법들의 리스트를 포함함
    syntax_categories = {
        "JOIN": ["JOIN", "INNER JOIN", "LEFT JOIN", "RIGHT JOIN", "OUTER JOIN"],  # 테이블 조인 관련 구문들을 포함함
        "CROSS_JOIN": ["CROSS JOIN"],  # 모든 행의 조합을 생성하는 카티션 곱 조인을 별도 카테고리로 분리함
        "SET_OPERATIONS": ["UNION", "INTERSECT", "EXCEPT"],  # 집합 연산자들을 포함함
        "AGG_FUNCTIONS": ["SUM", "COUNT", "AVG", "MIN", "MAX"],  # 집계 함수들을 포함함
        "CONDITIONS": ["WHERE", "HAVING"],  # 조건절을 포함함
        "WINDOW": ["WINDOW FUNCTION"],  # 윈도우 함수를 별도 카테고리로 분리함
        "OTHERS": ["GROUP BY", "ORDER BY", "SUBQUERY", "CASE", "DISTINCT", "LIMIT",
                   "OFFSET", "LIKE", "IN", "BETWEEN", "EXISTS", "NOT EXISTS", "CTE"]  # 기타 다양한 SQL 문법들을 포함함
    }

    # 난이도에 따라 선택할 SQL 문법 개수의 범위를 정의함
    # 난이도가 높을수록 더 많은 문법이 선택되도록 최소값과 최대값을 조정함
    syntax_count_ranges = {
        1: (1, 2),     # 난이도 1: 최소 1개, 최대 2개 문법 선택함
        2: (2, 3),     # 난이도 2: 최소 2개, 최대 3개 문법 선택함
        3: (3, 4),     # 난이도 3: 최소 3개, 최대 4개 문법 선택함
        4: (5, 6),     # 난이도 4: 최소 5개, 최대 6개 문법 선택함 (최소값 증가시킴)
        5: (7, 8)      # 난이도 5: 최소 7개, 최대 8개 문법 선택함 (최소값과 최대값 모두 증가시킴)
    }

    # 주어진 난이도에 해당하는 범위에서 무작위로 문법 개수를 선택함
    # *을 사용하여 튜플의 값을 함수 인자로 언패킹함
    max_syntax_count = random.randint(*syntax_count_ranges[difficulty])
    
    # 중복 없이 문법을 저장하기 위해 집합(set) 자료구조를 사용함
    selected_syntax = set()

    # 난이도가 1~2인 경우 단순한 문법만 선택함
    if difficulty <= 2:
        # 간단한 문법 목록에서 무작위로 선택함
        selected_syntax.update(random.sample(
            ["WHERE", "ORDER BY", "LIMIT", "LIKE", "IN", "BETWEEN"], 
            max_syntax_count
        ))
        # 선택된 문법들을 리스트로 변환하여 반환함
        return list(selected_syntax)

    # 난이도가 3 이상부터 복잡한 문법을 포함시킴
    # 집합 연산자(UNION, INTERSECT, EXCEPT) 선택 확률을 난이도에 따라 차등 적용함
    set_op_prob = 0.3 if difficulty == 3 else 0.6 if difficulty == 4 else 0.9
    # 난수를 생성하여 확률에 따라 집합 연산자를 선택함
    if random.random() < set_op_prob:
        selected_syntax.add(random.choice(syntax_categories["SET_OPERATIONS"]))

    # JOIN 문법 선택 개수를 난이도에 따라 차등화함
    join_count_range = {
        3: (0, 1),  # 난이도 3: 0-1개 JOIN을 선택함 (JOIN이 없을 수도 있음)
        4: (1, 2),  # 난이도 4: 1-2개 JOIN을 선택함 (최소 1개 JOIN을 보장함)
        5: (2, 2)   # 난이도 5: 반드시 2개의 JOIN을 선택함
    }
    
    # 난이도가 3 이상인 경우에만 JOIN 문법을 고려함
    if difficulty >= 3:
        # 해당 난이도의 JOIN 개수 범위를 가져옴
        join_min, join_max = join_count_range[difficulty]
        # 범위 내에서 무작위로 JOIN 개수를 선택함
        join_count = random.randint(join_min, join_max)
        # 선택된 개수만큼 JOIN 문법을 추가함
        if join_count > 0:
            selected_syntax.update(random.sample(syntax_categories["JOIN"], join_count))

    # CROSS JOIN 추가 확률을 난이도에 따라 차등화함
    # 난이도 4 미만에서는 CROSS JOIN을 사용하지 않음
    cross_join_prob = 0 if difficulty < 4 else 0.4 if difficulty == 4 else 0.7
    # 난수를 생성하여 확률에 따라 CROSS JOIN을 선택함
    if random.random() < cross_join_prob:
        selected_syntax.add("CROSS JOIN")

    # GROUP BY + HAVING + 집계함수 조합 확률을 난이도에 따라 차등화함
    agg_prob = 0.5 if difficulty == 3 else 0.7 if difficulty == 4 else 0.9
    # 난수를 생성하여 확률에 따라 GROUP BY와 관련 문법을 선택함
    if random.random() < agg_prob:
        # GROUP BY 문법 추가함
        selected_syntax.add("GROUP BY")
        # 난이도가 4 이상이거나 50% 확률로 HAVING 문법을 추가함
        if difficulty >= 4 or random.random() < 0.5:
            selected_syntax.add("HAVING")
        
        # 난이도 5에서는 50% 확률로 집계함수를 여러 개(1-2개) 추가함
        if difficulty == 5 and random.random() < 0.5:
            agg_count = random.randint(1, 2)
            # 여러 개의 집계함수를 선택하여 추가함
            selected_syntax.update(random.sample(syntax_categories["AGG_FUNCTIONS"], agg_count))
        else:
            # 하나의 집계함수만 선택하여 추가함
            selected_syntax.add(random.choice(syntax_categories["AGG_FUNCTIONS"]))

    # WINDOW FUNCTION 추가 확률을 난이도에 따라 차등화함
    window_prob = 0 if difficulty < 3 else 0.3 if difficulty == 3 else 0.6 if difficulty == 4 else 0.8
    # 난수를 생성하여 확률에 따라 WINDOW FUNCTION을 선택함
    if random.random() < window_prob:
        selected_syntax.add("WINDOW FUNCTION")
    # WINDOW FUNCTION이 선택되지 않았을 때 50% 확률로 DISTINCT를 대신 추가함
    elif random.random() < 0.5 and "WINDOW FUNCTION" not in selected_syntax:
        selected_syntax.add("DISTINCT")

    # EXISTS/NOT EXISTS 추가 확률을 난이도에 따라 차등화함
    exists_prob = 0.3 if difficulty == 3 else 0.5 if difficulty == 4 else 0.7
    # 난수를 생성하여 확률에 따라 EXISTS 또는 NOT EXISTS를 선택함
    if random.random() < exists_prob:
        selected_syntax.add(random.choice(["EXISTS", "NOT EXISTS"]))

    # 목표한 문법 개수에 도달하지 못한 경우 나머지 문법을 추가함
    remaining_count = max_syntax_count - len(selected_syntax)
    if remaining_count > 0:
        # 아직 선택되지 않은 OTHERS 카테고리의 문법들을 찾음
        additional_syntax = [s for s in syntax_categories["OTHERS"] if s not in selected_syntax]
        # 필요한 개수만큼 추가 문법을 선택하여 추가함
        # min() 함수를 사용하여 남은 문법 수와 필요한 개수 중 작은 값을 선택함
        selected_syntax.update(random.sample(
            additional_syntax, 
            min(len(additional_syntax), remaining_count)
        ))

    # 최종적으로 선택된 문법들을 리스트로 변환하여 반환함
    return list(selected_syntax)

# 사용 예시 코드
for i in range(2):
    # 테스트 번호를 출력함
    print(f"{i+1}번째 테스트")
    # 각 난이도(1~5)에 대해 예시를 생성하고 출력함
    for difficulty in range(1, 6):
        print(f"난이도 {difficulty}의 예시:", select_realistic_main_syntax(difficulty))
    # 구분선을 출력하여 테스트 결과를 구분함
    print("--------------------------------")

1번째 테스트
난이도 1의 예시: ['ORDER BY']
난이도 2의 예시: ['LIMIT', 'WHERE']
난이도 3의 예시: ['EXCEPT', 'DISTINCT', 'IN']
난이도 4의 예시: ['INNER JOIN', 'HAVING', 'JOIN', 'GROUP BY', 'EXISTS', 'COUNT']
난이도 5의 예시: ['INNER JOIN', 'UNION', 'HAVING', 'RIGHT JOIN', 'GROUP BY', 'MAX', 'EXISTS', 'CROSS JOIN', 'SUM']
--------------------------------
2번째 테스트
난이도 1의 예시: ['LIKE', 'IN']
난이도 2의 예시: ['LIKE', 'BETWEEN', 'IN']
난이도 3의 예시: ['MAX', 'GROUP BY', 'OUTER JOIN', 'DISTINCT']
난이도 4의 예시: ['DISTINCT', 'HAVING', 'JOIN', 'GROUP BY', 'SUM']
난이도 5의 예시: ['LIMIT', 'UNION', 'OUTER JOIN', 'DISTINCT', 'RIGHT JOIN', 'GROUP BY', 'NOT EXISTS', 'BETWEEN']
--------------------------------


### 📌  함께 쓰이기 어려운 SQL Syntax 조합 정리  
아래는 대표적으로 현실에서 거의 같이 쓰이지 않거나, 같이 쓰이면 어색한 조합의 예시입니다.

- 집합 연산자 간 중복 사용:
    - ["UNION", "INTERSECT", "EXCEPT"] → 한 번에 하나만 선택

- JOIN 종류 간 중복 사용 (일반적으로 여러 JOIN 유형을 한꺼번에 무작위로 사용하는 경우 드물며, 특히 CROSS JOIN은 특정한 목적에서만 사용됨):
    - ["JOIN", "INNER JOIN", "LEFT JOIN", "RIGHT JOIN", "OUTER JOIN", "CROSS JOIN"]
        → 현실적으로는 JOIN은 한 번의 쿼리에 최대 2~3가지로 제한하는 게 좋음 (특히 CROSS JOIN은 따로 구분)

- Aggregation(HAVING)과 Aggregation 함수(SUM, COUNT 등)의 관계성:
    - ["HAVING"]은 ["GROUP BY"] 없이는 잘 사용되지 않으며, 또한 Aggregation 함수("SUM","COUNT","AVG","MIN","MAX") 중 최소 하나는 반드시 사용되어야 함
Window Functions은 DISTINCT와 함께 쓰이는 경우가 드뭄
    - EXISTS와 NOT EXISTS는 함께 사용되지 않음 (일반적으로 쿼리 의도가 다르므로 동시 사용X)

In [7]:
system_prompt = "당신은 Text-to-SQL 데이터셋 생성을 위한 전문 AI 도우미입니다."

user_prompt = """## 목표
주어진 조건에 맞는 자연스러운 한국어 질문과 해당하는 SQL 쿼리를 생성하세요.

## 출력 형식
다음 JSON 형식으로 데이터를 생성하세요:
{{
  "TEXT": "사용자 질문 (한국어)",
  "MySQL": "SQL 쿼리",
  "Schema": "DB 및 테이블 정의",
  "Difficulty": 1-5 숫자,
  "JOIN Count": 숫자,
  "DB Count": 숫자,
  "Type": "SQL 타입",
  "Main Syntax": ["사용된 주요 문법"],
  "Domain": "분야",
  "Domain Description": "분야 설명",
  "SQL Complexity": "쿼리 복잡성",
  "SQL Complexity Description": "복잡성 설명",
  "SQL Task Type": "SQL 작업 유형",
  "SQL Task Type Description": "작업 유형 설명",
  "SQL Explanation": "SQL 기능 설명"
}}

## 출력 예시
{{
  "TEXT": "김철수 교수가 가르치는 과목의 이름과 학점을 알려줘.",
  "MySQL": "SELECT 과목이름, 학점 FROM 과목 WHERE 교수명 = '김철수';",
  "Schema": "DB: university_db\nTABLE DDL:\nCREATE TABLE `과목` (\n    과목ID INT NOT NULL,\n    과목이름 VARCHAR(100),\n    교수명 VARCHAR(100),\n    학점 INT,\n    PRIMARY KEY (과목ID)\n)",
  "Difficulty": 2,
  "JOIN Count": 0,
  "DB Count": 1,
  "Type": "SELECT",
  "Main Syntax": ["WHERE"],
  "Domain": "교육",
  "Domain Description": "대학교에서 운영되는 과목, 교수, 학생의 학사정보와 관련된 데이터입니다.",
  "SQL Complexity": "basic SQL",
  "SQL Complexity Description": "단순한 SELECT 문을 사용하는 기본적인 SQL입니다.",
  "SQL Task Type": "기본 데이터 조회",
  "SQL Task Type Description": "기본적인 데이터 조회를 수행합니다.",
  "SQL Explanation": "이 쿼리는 김철수 교수가 가르치는 과목 이름과 학점을 조회합니다."
}}

## 생성 규칙
1. **Schema**: 반드시 DB 이름과 테이블 DDL 정의를 포함
2. **TEXT**: 실제 한국인 사용자가 물어볼 만한 자연스러운 질문
3. **MySQL**: Schema에 맞는 정확한 SQL 쿼리 작성
4. **Difficulty**: 질의 복잡성에 따라 1-5로 평가 (주요 요소: JOIN, 집계, 서브쿼리, 조건절)
5. **Main Syntax**: 사용된 주요 SQL 문법 리스트

## SQL 문법 규칙
- JOIN 유형은 한 쿼리에 최대 2개까지만 사용
- 집계 함수 사용 시 반드시 GROUP BY 또는 HAVING과 함께 사용
- HAVING은 GROUP BY 및 집계 함수와 함께만 사용
- 집합 연산자(UNION, INTERSECT, EXCEPT)는 한 쿼리에 하나만 사용
- EXISTS와 NOT EXISTS는 둘 중 하나만 사용
- WINDOW FUNCTION과 DISTINCT는 같은 쿼리에서 함께 사용하지 않음

## 현재 생성 조건
- Difficulty: {difficulty}
- Schema Table Count: {difficulty}
- Domain: {domain}
- SQL Task Type: {sql_task_type}
- SQL Complexity: {sql_complexity}
- Type: {sql_type}
- Main Syntax: {main_syntax}

# Text-to-SQL 데이터셋 생성 작업
Plan : 주어진 생성 조건에 맞게 Schema와 Query를 생성하기 위해 계획 작성 
Schema : Plan에 맞게 Schema를 작성
Qeustion : Plan과 Schema에 맞는 Qeustion 작성  
Query : Question을 수행하기 위한 Query 작성 

## 평가 과정
1. 논리적으로 어떻게 쿼리를 작성되었는지 이유 작성(Reason)(한국어로 작성)
2. 최종적으로 쿼리가 조건에 맞게 생성되었는지 평가(Result)하세요 : True/False로 표시

위 조건을 모두 충족하는 고품질 데이터셋 항목을 생성하세요.
"""

In [8]:
import itertools  # 다양한 반복 도구를 제공하는 모듈을 가져옴. 여기서는 product 함수를 사용하기 위함임

# 다양한 난이도 레벨을 리스트로 정의함. SQL 문제의 난이도를 1부터 5까지 표현함
difficulty_levels = [1, 2, 3, 4, 5]

# category_metadata 딕셔너리에서 필요한 카테고리 정보를 추출하여 각 변수에 저장함
# 이 변수들은 SQL 문제 생성에 필요한 여러 차원의 분류 기준을 담고 있음
domains = category_metadata["Domain"]  # SQL 문제가 적용될 수 있는 비즈니스/응용 영역을 저장함 (예: 금융, 인사관리, 판매 등)
task_types = category_metadata["SQL Task Type"]  # SQL 작업 유형을 저장함 (예: 쿼리, 업데이트, 삭제 등)
complexities = category_metadata["SQL Complexity"]  # SQL 구문 복잡성을 저장함 (예: 단순, 보통, 복잡 등)
types = category_metadata["Type"]  # 문제 유형을 저장함 (예: 객관식, 주관식, 코딩 등)

# itertools.product 함수를 사용하여 모든 가능한 조합을 생성함
# product는 수학적 카티션 곱(Cartesian product)을 구현하여 모든 입력 집합의 요소를 조합함
# 예를 들어, 난이도 1-5, 도메인 3개, 작업 유형 4개, 복잡성 3개, 문제 유형 2개가 있다면
# 5 * 3 * 4 * 3 * 2 = 360개의 조합이 생성됨
all_combinations = list(itertools.product(
    difficulty_levels,
    domains,
    task_types,
    complexities,
    types
))

# 생성된 총 조합의 개수를 계산하고 출력함
# 이는 생성 가능한 모든 SQL 문제 유형의 개수를 의미함
print(f"총 가능한 조합 수: {len(all_combinations)}개")

총 가능한 조합 수: 36960개


In [9]:
# 난이도 레벨을 제외하고 나머지 4개 차원(도메인, 작업 유형, 복잡성, 유형)의 모든 조합을 생성함
# 이는 기존 조합(36960개)이 너무 많아 계산 효율성을 위해 난이도를 분리한 접근법임
# 각 조합은 (도메인, 작업 유형, 복잡성, 유형)의 튜플 형태로 생성됨
all_combinations = list(itertools.product(
    domains,  # SQL 문제가 적용될 수 있는 비즈니스/응용 영역 리스트임
    task_types,  # SQL 작업 유형(조회, 업데이트, 삭제 등)의 리스트임
    complexities,  # SQL 구문 복잡성 수준을 나타내는 리스트임
    types  # 문제 형식(객관식, 주관식, 코딩 등)을 나타내는 리스트임
))

# 난이도를 제외한 총 조합의 수를 계산하고 출력함
# 원래 조합 수가 36960개였다면, 난이도 레벨이 5개일 경우
# 36960 ÷ 5 = 7392개의 조합이 생성됨을 예상할 수 있음
print(f"난이도 제외 총 가능한 조합 수: {len(all_combinations)}개")

난이도 제외 총 가능한 조합 수: 7392개


In [10]:
from collections import Counter  # 요소의 개수를 세는 특수 딕셔너리 클래스를 가져옴. 다양한 통계 작업을 위해 준비된 것으로 추정됨

# 각 난이도에 할당할 비율을 딕셔너리 형태로 설정함
# 키는 난이도(1-5), 값은 각 난이도가 전체 조합에서 차지할 비율임
difficulty_distribution = {
    1: 0.03,  # 전체 조합 중 3%를 난이도 1로 할당함
    2: 0.03,  # 전체 조합 중 3%를 난이도 2로 할당함
    3: 0.03,  # 전체 조합 중 3%를 난이도 3로 할당함
    4: 0.05,  # 전체 조합 중 5%를 난이도 4로 할당함
    5: 0.10   # 전체 조합 중 10%를 난이도 5로 할당함
}

# 전체 조합에서 실제로 샘플링할 총 비율을 정의함
# 이는 모든 난이도 비율의 합계와 같음 (0.03 + 0.03 + 0.03 + 0.05 + 0.10 = 0.24)
# 즉, 모든 가능한 조합 중 24%만 추출하여 사용할 계획임
sampling_ratio = 0.24  

# 전체 조합 개수와 샘플링할 총 개수를 계산함
# 전체 조합 개수에 샘플링 비율을 곱하여 실제로 생성할 문제 개수를 결정함
total_combinations = len(all_combinations)  # 전체 조합 개수를 가져옴
sampled_total = int(total_combinations * sampling_ratio)  # 샘플링할 총 개수를 계산함
# 계산 결과를 출력하여 확인함
print(f"전체 조합 개수: {total_combinations}, 샘플링할 개수: {sampled_total}")

# 각 난이도별로 실제 샘플링할 개수를 계산함
# 비율의 총합을 계산하여 상대적 비중을 구하는 과정임
total_ratio = sum(difficulty_distribution.values())  # 모든 난이도 비율의 합계를 계산함
difficulty_counts = {}  # 각 난이도별 할당할 개수를 저장할 딕셔너리를 초기화함
for diff, ratio in difficulty_distribution.items():
    # 각 난이도의 상대적 비중을 계산함 (해당 난이도 비율 / 전체 비율)
    relative_ratio = ratio / total_ratio
    # 총 샘플링 개수에 상대적 비중을 곱하여 해당 난이도의 할당 개수를 계산함
    difficulty_counts[diff] = int(sampled_total * relative_ratio)

# 정수로 변환 과정에서 발생한 반올림 오차를 보정함
# 반올림으로 인해 총 합계가 목표보다 적을 경우 난이도 5에 나머지를 추가함
allocated = sum(difficulty_counts.values())  # 현재까지 할당된 총 개수를 계산함
if allocated < sampled_total:  # 목표보다 적게 할당되었다면
    # 부족한 수는 난이도 5에 추가하여 총 샘플링 개수를 정확히 맞춤
    difficulty_counts[5] += (sampled_total - allocated)

# 전체 조합 리스트를 무작위로 섞음
# 이는 특정 패턴이나 순서에 따른 편향을 방지하기 위함임
random.shuffle(all_combinations)

# 난이도별로 조합을 추출하여 최종 조합 리스트를 생성함
final_combinations = []  # 최종 선택된 조합들을 저장할 리스트를 초기화함
remaining_idx = 0  # 다음에 처리할 조합의 인덱스를 추적하는 변수를 초기화함

# 각 난이도별로 할당된 개수만큼 조합을 추출함
for diff, count in difficulty_counts.items():  # 각 난이도와 할당 개수를 순회함
    for _ in range(count):  # 해당 난이도에 할당된 개수만큼 반복함
        if remaining_idx >= total_combinations:  # 인덱스가 범위를 벗어나면 중단함
            break
        comb = all_combinations[remaining_idx]  # 현재 인덱스의 조합을 가져옴
        # 난이도를 조합의 첫 요소로 추가하여 최종 조합 리스트에 추가함
        # *comb는 튜플을 언패킹하여 각 요소를 개별 인자로 전달함
        final_combinations.append((diff, *comb))
        remaining_idx += 1  # 다음 조합으로 인덱스를 증가시킴

# 난이도별 할당 결과를 확인하여 출력함
# Counter 클래스를 사용하여 각 난이도별 개수를 계산함
counter = Counter([c[0] for c in final_combinations])  # 첫 번째 요소(난이도)만 추출하여 개수를 계산함
print("난이도별 할당된 개수:", counter)  # 난이도별 할당된 개수를 출력함
print(f"총 생성된 조합: {len(final_combinations)}/{sampled_total} (목표)")  # 총 생성된 조합 개수와 목표를 비교하여 출력함

# 최종 프롬프트 생성 함수를 정의함
# 이 함수는 조합 정보를 받아 실제 SQL 문제 생성을 위한 프롬프트와 메타데이터를 반환함
def create_final_prompt(combination):
    # 조합에서 각 요소를 개별 변수로 추출함
    difficulty, domain, sql_task_type, sql_complexity, sql_type = combination
    # 난이도에 맞는 SQL 문법 조합을 생성함 (이전에 정의된 함수 사용)
    main_syntax = select_realistic_main_syntax(difficulty)

    # 미리 정의된 템플릿(user_prompt)에 추출한 정보를 삽입하여 최종 프롬프트를 생성함
    prompt = user_prompt.format(
        difficulty=difficulty,  # 난이도 정보를 삽입함
        domain=domain,  # 도메인 정보를 삽입함
        sql_task_type=sql_task_type,  # SQL 작업 유형 정보를 삽입함
        sql_complexity=sql_complexity,  # SQL 복잡성 정보를 삽입함
        sql_type=sql_type,  # 문제 유형 정보를 삽입함
        main_syntax=main_syntax  # 생성된 SQL 문법 조합을 삽입함
    )

    # 프롬프트 생성에 사용된 모든 정보를 딕셔너리로 구성하여 메타데이터로 반환함
    # 이는 추후 분석이나 추적을 위한 목적으로 사용될 수 있음
    info = {
        "difficulty": difficulty,  # 난이도 정보를 저장함
        "domain": domain,  # 도메인 정보를 저장함
        "sql_task_type": sql_task_type,  # SQL 작업 유형 정보를 저장함
        "sql_complexity": sql_complexity,  # SQL 복잡성 정보를 저장함
        "sql_type": sql_type,  # 문제 유형 정보를 저장함
        "main_syntax": main_syntax  # 생성된 SQL 문법 조합 정보를 저장함
    }
    # 생성된 프롬프트와 메타데이터를 튜플로 반환함
    return prompt, info

전체 조합 개수: 7392, 샘플링할 개수: 1774
난이도별 할당된 개수: Counter({5: 742, 4: 369, 1: 221, 2: 221, 3: 221})
총 생성된 조합: 1774/1774 (목표)


In [11]:
# 의도한대로 만들어졌는지 확인 
selected_combination = final_combinations[-100]  # 또는 다른 인덱스
convert_selected_combination = [ create_final_prompt(prompt_input_data) for prompt_input_data in final_combinations]

convert_selected_combination[-2:]

[('## 목표\n주어진 조건에 맞는 자연스러운 한국어 질문과 해당하는 SQL 쿼리를 생성하세요.\n\n## 출력 형식\n다음 JSON 형식으로 데이터를 생성하세요:\n{\n  "TEXT": "사용자 질문 (한국어)",\n  "MySQL": "SQL 쿼리",\n  "Schema": "DB 및 테이블 정의",\n  "Difficulty": 1-5 숫자,\n  "JOIN Count": 숫자,\n  "DB Count": 숫자,\n  "Type": "SQL 타입",\n  "Main Syntax": ["사용된 주요 문법"],\n  "Domain": "분야",\n  "Domain Description": "분야 설명",\n  "SQL Complexity": "쿼리 복잡성",\n  "SQL Complexity Description": "복잡성 설명",\n  "SQL Task Type": "SQL 작업 유형",\n  "SQL Task Type Description": "작업 유형 설명",\n  "SQL Explanation": "SQL 기능 설명"\n}\n\n## 출력 예시\n{\n  "TEXT": "김철수 교수가 가르치는 과목의 이름과 학점을 알려줘.",\n  "MySQL": "SELECT 과목이름, 학점 FROM 과목 WHERE 교수명 = \'김철수\';",\n  "Schema": "DB: university_db\nTABLE DDL:\nCREATE TABLE `과목` (\n    과목ID INT NOT NULL,\n    과목이름 VARCHAR(100),\n    교수명 VARCHAR(100),\n    학점 INT,\n    PRIMARY KEY (과목ID)\n)",\n  "Difficulty": 2,\n  "JOIN Count": 0,\n  "DB Count": 1,\n  "Type": "SELECT",\n  "Main Syntax": ["WHERE"],\n  "Domain": "교육",\n  "Domain Description": "대학교에서 운영되는 과목, 교수

In [12]:
from pydantic import BaseModel  # 데이터 모델 정의와 유효성 검증을 위한 Pydantic 라이브러리의 BaseModel 클래스를 가져옴

# JSON 응답 형식을 정의하는 Pydantic 모델 클래스를 생성함
# 이는 API 응답을 구조화하고 타입 검증을 수행하는 역할을 함
class json_output(BaseModel): 
    Plan : str    # SQL 쿼리 계획 또는 접근 방식을 담는 문자열 필드임
    Schema : str  # 데이터베이스 스키마 정보를 담는 문자열 필드임
    Qeustion : str  # 생성된 SQL 문제의 질문 텍스트를 담는 문자열 필드임 (Qeustion이라고 오타가 있음, Question이 정확함)
    Query : str   # 문제에 대한 SQL 쿼리 솔루션을 담는 문자열 필드임
    Reason : str  # 쿼리가 작동하는 방식에 대한 설명이나 이유를 담는 문자열 필드임
    Result : bool  # 쿼리가 성공적으로 실행되었는지 여부를 나타내는 불리언 필드임

# 환경 변수 파일을 로드함. 이는 API 키와 같은 민감한 정보를 소스 코드에 직접 포함시키지 않기 위함임
load_dotenv("./credit-env")

# OpenAI API 클라이언트를 초기화함. 환경 변수에서 API 키를 가져와 사용함
# 이를 통해 GPT-4o 모델에 안전하게 접근할 수 있음
client = OpenAI(api_key=os.getenv("SELF_OPENAI_API_KEY"))

# 최종 프롬프트를 입력으로 받아 GPT-4o 모델을 사용하여 SQL 쿼리와 스키마를 생성하는 함수를 정의함
def Query_Schema_generation(final_prompt):
    # OpenAI API의 chat.completions.parse 메서드를 호출하여 구조화된 JSON 응답을 요청함
    # parse 메서드는 응답을 특정 형식(여기서는 json_output 클래스)으로 파싱하도록 함
    completion = client.beta.chat.completions.parse(
        model="gpt-4o",  # GPT-4o 모델을 사용함
        messages=[
            {"role": "system", "content": f"{system_prompt}"},  # 시스템 프롬프트를 설정하여 모델의 동작 방식을 지정함
            {
                "role": "user",  # 사용자 메시지 역할을 지정함
                "content": f"{final_prompt})"  # 최종 프롬프트를 사용자 메시지로 전달함 (괄호가 닫히지 않은 것은 오타일 수 있음)
            }
        ],
        response_format=json_output,  # 응답을 json_output 클래스 형식으로 파싱하도록 지정함
        temperature=0.0  # 온도를 0으로 설정하여 일관된 응답을 얻음 (낮은 온도는 더 결정적인 출력을 생성함)
    )

    # API 응답에서 필요한 데이터를 추출함
    # choices[0]는 첫 번째(유일한) 응답을 가리키며, message.parsed는 파싱된 JSON 객체를 반환함
    data_dict = completion.dict()["choices"][0]["message"]["parsed"]
    
    # 파싱된 데이터 딕셔너리를 반환함
    # 이 딕셔너리는 Plan, Schema, Qeustion, Query, Reason, Result 필드를 포함함
    return data_dict

In [13]:
# pqdm(Parallel Query Data Management)을 사용하여 여러 프로세스에서 동시에 작업을 처리하는 함수를 정의함
def process_all_combinations(convert_selected_combination, n_jobs=4):
    """
    모든 조합을 병렬로 처리하여 데이터셋 생성하고, input_prompt[1]과 결합
    
    Args:
        convert_selected_combination: 처리할 조합 목록 (각 항목은 [input_prompt[0], input_prompt[1]] 형태)
        n_jobs: 병렬 처리 작업 수
        
    Returns:
        input_prompt[1]과 결합된 데이터셋 항목 목록
    """
    # 각 조합의 첫 번째 요소(input_prompt[0])만 추출하여 리스트를 생성함
    # 이 요소는 실제 프롬프트 내용으로, Query_Schema_generation 함수에 전달될 입력값임
    input_info_list = [input_prompt[0] for input_prompt in convert_selected_combination]
    
    # 각 조합의 두 번째 요소(input_prompt[1])만 추출하여 리스트를 생성함
    # 이 요소는 메타데이터로, 최종 결과에 함께 저장될 정보임
    metadata_list = [input_prompt[1] for input_prompt in convert_selected_combination]
    
    # 처리할 조합 개수와 사용할 프로세스 수를 출력하여 현재 작업 상황을 알림
    print(f"{len(input_info_list)}개의 조합을 {n_jobs}개의 프로세스로 처리합니다...")
    
    # pqdm 함수를 사용하여 여러 프로세스에서 동시에 Query_Schema_generation 함수를 실행함
    # input_info_list의 각 항목이 Query_Schema_generation 함수의 인자로 전달됨
    # n_jobs 매개변수는 동시에 실행할 프로세스 수를 지정함
    # desc 매개변수는 진행 상황 표시줄에 표시될 설명 텍스트를 지정함
    processed_results = pqdm(input_info_list, Query_Schema_generation, n_jobs=n_jobs, desc="데이터셋 생성 중")
    
    # 처리 결과와 메타데이터를 결합하여 최종 결과 리스트를 생성함
    final_results = []
    # 처리된 각 결과에 대해 반복함
    for i, result in enumerate(processed_results):
        # 결과가 None이 아닌 경우만 처리함 (오류가 발생한 경우 None이 반환될 수 있음)
        if result is not None:
            # 딕셔너리 언패킹 구문(**dict)을 사용하여 결과와 메타데이터를 하나의 딕셔너리로 결합함
            # {**result, **metadata_list[i]}는 result의 모든 키-값 쌍과 metadata_list[i]의 모든 키-값 쌍을 포함하는 새 딕셔너리를 생성함
            combined_result = {**result, **metadata_list[i]}
            # 결합된 결과를 최종 결과 리스트에 추가함
            final_results.append(combined_result)
    
    # 성공적으로 처리된 항목 수와 전체 항목 수를 출력하여 처리 결과를 요약함
    print(f"{len(final_results)}/{len(input_info_list)} 항목 처리 완료.")
    # 최종 결과 리스트를 반환함
    return final_results

In [14]:
# 실제로 코드가 작동하는지 한번 확인
temp_combination = convert_selected_combination[:2]
temp_combination 

[('## 목표\n주어진 조건에 맞는 자연스러운 한국어 질문과 해당하는 SQL 쿼리를 생성하세요.\n\n## 출력 형식\n다음 JSON 형식으로 데이터를 생성하세요:\n{\n  "TEXT": "사용자 질문 (한국어)",\n  "MySQL": "SQL 쿼리",\n  "Schema": "DB 및 테이블 정의",\n  "Difficulty": 1-5 숫자,\n  "JOIN Count": 숫자,\n  "DB Count": 숫자,\n  "Type": "SQL 타입",\n  "Main Syntax": ["사용된 주요 문법"],\n  "Domain": "분야",\n  "Domain Description": "분야 설명",\n  "SQL Complexity": "쿼리 복잡성",\n  "SQL Complexity Description": "복잡성 설명",\n  "SQL Task Type": "SQL 작업 유형",\n  "SQL Task Type Description": "작업 유형 설명",\n  "SQL Explanation": "SQL 기능 설명"\n}\n\n## 출력 예시\n{\n  "TEXT": "김철수 교수가 가르치는 과목의 이름과 학점을 알려줘.",\n  "MySQL": "SELECT 과목이름, 학점 FROM 과목 WHERE 교수명 = \'김철수\';",\n  "Schema": "DB: university_db\nTABLE DDL:\nCREATE TABLE `과목` (\n    과목ID INT NOT NULL,\n    과목이름 VARCHAR(100),\n    교수명 VARCHAR(100),\n    학점 INT,\n    PRIMARY KEY (과목ID)\n)",\n  "Difficulty": 2,\n  "JOIN Count": 0,\n  "DB Count": 1,\n  "Type": "SELECT",\n  "Main Syntax": ["WHERE"],\n  "Domain": "교육",\n  "Domain Description": "대학교에서 운영되는 과목, 교수

In [15]:
# 병렬 처리 실행
results = process_all_combinations(temp_combination, n_jobs=2)

# 첫 번째 결과 출력
if results:
    print(json.dumps(results[0], indent=2, ensure_ascii=False))

2개의 조합을 2개의 프로세스로 처리합니다...


QUEUEING TASKS | 데이터셋 생성 중: 100%|██████████| 2/2 [00:00<00:00, 94.26it/s]
PROCESSING TASKS | 데이터셋 생성 중:   0%|          | 0/2 [00:00<?, ?it/s]

PROCESSING TASKS | 데이터셋 생성 중: 100%|██████████| 2/2 [00:06<00:00,  3.38s/it]
COLLECTING RESULTS | 데이터셋 생성 중: 100%|██████████| 2/2 [00:00<00:00, 26886.56it/s]

2/2 항목 처리 완료.
{
  "Plan": "공공 보건 분야에서 백업 및 복구 관리와 관련된 데이터를 다루는 시나리오를 설정합니다. 예를 들어, 특정 날짜 이전에 백업된 환자 기록을 삭제하는 작업을 수행할 수 있습니다. 이 작업은 DELETE 쿼리를 사용하여 수행되며, WHERE 절과 IN 절을 사용하여 특정 조건에 맞는 데이터를 삭제합니다.",
  "Schema": "DB: healthcare_db\nTABLE DDL:\nCREATE TABLE `환자기록` (\n    기록ID INT NOT NULL,\n    환자명 VARCHAR(100),\n    백업일 DATE,\n    PRIMARY KEY (기록ID)\n)",
  "Qeustion": "2023년 1월 1일 이전에 백업된 환자 기록을 삭제해줘.",
  "Query": "DELETE FROM 환자기록 WHERE 백업일 < '2023-01-01';",
  "Reason": "공공 보건 분야에서 백업 및 복구 관리는 중요한 작업입니다. 이 시나리오에서는 특정 날짜 이전에 백업된 환자 기록을 삭제하는 작업을 수행합니다. DELETE 쿼리를 사용하여 환자기록 테이블에서 백업일이 2023년 1월 1일 이전인 모든 기록을 삭제합니다. 이 작업은 WHERE 절을 사용하여 조건을 지정하고, IN 절은 사용되지 않았습니다. 따라서, 주어진 조건에 맞는 간단한 DELETE 쿼리를 작성하였습니다.",
  "Result": false,
  "difficulty": 1,
  "domain": "공공 보건",
  "sql_task_type": "백업 및 복구 관리",
  "sql_complexity": "multiple joins",
  "sql_type": "DELETE",
  "main_syntax": [
    "WHERE",
    "IN"
  ]
}


In [16]:
results

[{'Plan': '공공 보건 분야에서 백업 및 복구 관리와 관련된 데이터를 다루는 시나리오를 설정합니다. 예를 들어, 특정 날짜 이전에 백업된 환자 기록을 삭제하는 작업을 수행할 수 있습니다. 이 작업은 DELETE 쿼리를 사용하여 수행되며, WHERE 절과 IN 절을 사용하여 특정 조건에 맞는 데이터를 삭제합니다.',
  'Schema': 'DB: healthcare_db\nTABLE DDL:\nCREATE TABLE `환자기록` (\n    기록ID INT NOT NULL,\n    환자명 VARCHAR(100),\n    백업일 DATE,\n    PRIMARY KEY (기록ID)\n)',
  'Qeustion': '2023년 1월 1일 이전에 백업된 환자 기록을 삭제해줘.',
  'Query': "DELETE FROM 환자기록 WHERE 백업일 < '2023-01-01';",
  'Reason': '공공 보건 분야에서 백업 및 복구 관리는 중요한 작업입니다. 이 시나리오에서는 특정 날짜 이전에 백업된 환자 기록을 삭제하는 작업을 수행합니다. DELETE 쿼리를 사용하여 환자기록 테이블에서 백업일이 2023년 1월 1일 이전인 모든 기록을 삭제합니다. 이 작업은 WHERE 절을 사용하여 조건을 지정하고, IN 절은 사용되지 않았습니다. 따라서, 주어진 조건에 맞는 간단한 DELETE 쿼리를 작성하였습니다.',
  'Result': False,
  'difficulty': 1,
  'domain': '공공 보건',
  'sql_task_type': '백업 및 복구 관리',
  'sql_complexity': 'multiple joins',
  'sql_type': 'DELETE',
  'main_syntax': ['WHERE', 'IN']},
 {'Plan': '공공 보건 분야에서 특정 조건에 맞는 데이터를 삭제하는 쿼리를 작성합니다. 주어진 조건에 따라 하나의 테이블을 사용하고, IN 문법을 활용하여 데이터를 삭제합니다. 데이터 삭제 작업이므로 DELE

In [17]:
len(convert_selected_combination) / 2

887.0

In [18]:
# 병렬 처리 실행
results_all1 = process_all_combinations(convert_selected_combination[:887], n_jobs=5)

# 첫 번째 결과 출력
if results_all1:
    print(json.dumps(results_all1[0], indent=2, ensure_ascii=False))

887개의 조합을 5개의 프로세스로 처리합니다...


QUEUEING TASKS | 데이터셋 생성 중: 100%|██████████| 887/887 [00:00<00:00, 17524.84it/s]
PROCESSING TASKS | 데이터셋 생성 중:   0%|          | 0/887 [00:00<?, ?it/s]

PROCESSING TASKS | 데이터셋 생성 중: 100%|██████████| 887/887 [21:36<00:00,  1.46s/it]
COLLECTING RESULTS | 데이터셋 생성 중: 100%|██████████| 887/887 [00:00<00:00, 647692.84it/s]

887/887 항목 처리 완료.
{
  "Plan": "공공 보건 분야에서 백업 및 복구 관리를 위한 DELETE 쿼리를 작성합니다. 주어진 조건에 따라 단일 테이블을 사용하고, WHERE와 IN 문법을 활용하여 특정 조건에 맞는 데이터를 삭제하는 쿼리를 생성합니다.",
  "Schema": "DB: public_health_db\nTABLE DDL:\nCREATE TABLE `백신_접종_기록` (\n    접종ID INT NOT NULL,\n    주민등록번호 VARCHAR(13),\n    백신종류 VARCHAR(50),\n    접종일 DATE,\n    접종기관 VARCHAR(100),\n    PRIMARY KEY (접종ID)\n)",
  "Qeustion": "주민등록번호가 '123456-7890123'인 사람의 백신 접종 기록을 삭제해줘.",
  "Query": "DELETE FROM 백신_접종_기록 WHERE 주민등록번호 IN ('123456-7890123');",
  "Reason": "주어진 조건에 따라 주민등록번호가 특정 값인 사람의 백신 접종 기록을 삭제하는 쿼리를 작성했습니다. DELETE 문을 사용하여 특정 조건에 맞는 데이터를 삭제하며, WHERE 절과 IN 연산자를 사용하여 주민등록번호가 '123456-7890123'인 레코드를 선택했습니다. 이는 단일 테이블에서 특정 조건에 맞는 데이터를 삭제하는 간단한 작업입니다.",
  "Result": true,
  "difficulty": 1,
  "domain": "공공 보건",
  "sql_task_type": "백업 및 복구 관리",
  "sql_complexity": "multiple joins",
  "sql_type": "DELETE",
  "main_syntax": [
    "WHERE",
    "IN"
  ]
}


In [21]:
df1 = pd.DataFrame(results_all1)
df1.head(2)


,Plan,Schema,Qeustion,Query,Reason,Result,difficulty,domain,sql_task_type,sql_complexity,sql_type,main_syntax
0,공공 보건 분야에서 백업 및 복구 관리를 위한 DELETE 쿼리를 작성합니다. 주어...,DB: public_health_db\nTABLE DDL:\nCREATE TABLE...,주민등록번호가 '123456-7890123'인 사람의 백신 접종 기록을 삭제해줘.,DELETE FROM 백신_접종_기록 WHERE 주민등록번호 IN ('123456-...,주어진 조건에 따라 주민등록번호가 특정 값인 사람의 백신 접종 기록을 삭제하는 쿼리...,True,1,공공 보건,백업 및 복구 관리,multiple joins,DELETE,"[WHERE, IN]"
1,공공 보건 분야에서 특정 조건에 맞는 데이터를 삭제하는 쿼리를 작성합니다. 주어진 ...,DB: public_health_db\nTABLE DDL:\nCREATE TABLE...,나이가 60세 이상인 환자들의 기록을 삭제해줘.,DELETE FROM 환자 WHERE 나이 IN (SELECT 나이 FROM 환자 ...,나이가 60세 이상인 환자들의 기록을 삭제하기 위해 DELETE 문을 사용합니다. ...,True,1,공공 보건,데이터 삭제,aggregation,UPDATE,[IN]


In [22]:
df1.to_csv("results_all1.csv", index=False)

In [23]:
# 병렬 처리 실행
results_all2 = process_all_combinations(convert_selected_combination[887:], n_jobs=5)

# 첫 번째 결과 출력
if results_all2:
    print(json.dumps(results_all2[0], indent=2, ensure_ascii=False))

887개의 조합을 5개의 프로세스로 처리합니다...


QUEUEING TASKS | 데이터셋 생성 중: 100%|██████████| 887/887 [00:00<00:00, 15760.38it/s]
PROCESSING TASKS | 데이터셋 생성 중: 100%|██████████| 887/887 [29:27<00:00,  1.99s/it]
COLLECTING RESULTS | 데이터셋 생성 중: 100%|██████████| 887/887 [00:00<00:00, 667686.23it/s]

887/887 항목 처리 완료.
{
  "Plan": "보험 도메인에서 고객의 보험 청구 데이터를 관리하는 테이블을 설정하고, 특정 조건에 맞는 데이터를 조회하는 쿼리를 작성한다. 고객 테이블, 보험 청구 테이블, 보험 상품 테이블, 보험사 테이블을 생성하고, 고객의 보험 청구 내역 중 가장 최근 청구 내역을 조회하는 쿼리를 작성한다. 이 쿼리는 서브쿼리와 윈도우 함수를 사용하여 가장 최근 청구 내역을 찾고, INNER JOIN을 통해 관련 정보를 결합한다.",
  "Schema": "DB: insurance_db\nTABLE DDL:\nCREATE TABLE `고객` (\n    고객ID INT NOT NULL,\n    이름 VARCHAR(100),\n    나이 INT,\n    PRIMARY KEY (고객ID)\n);\n\nCREATE TABLE `보험청구` (\n    청구ID INT NOT NULL,\n    고객ID INT,\n    보험상품ID INT,\n    청구금액 DECIMAL(10, 2),\n    청구일 DATE,\n    PRIMARY KEY (청구ID),\n    FOREIGN KEY (고객ID) REFERENCES 고객(고객ID)\n);\n\nCREATE TABLE `보험상품` (\n    보험상품ID INT NOT NULL,\n    상품명 VARCHAR(100),\n    보험사ID INT,\n    PRIMARY KEY (보험상품ID)\n);\n\nCREATE TABLE `보험사` (\n    보험사ID INT NOT NULL,\n    보험사명 VARCHAR(100),\n    PRIMARY KEY (보험사ID)\n);",
  "Qeustion": "가장 최근에 보험 청구를 한 고객의 이름과 청구 금액을 알려줘.",
  "Query": "SELECT 고객.이름, 보험청구.청구금액 \nFROM 보험청구 \nINNER JOIN 고객 ON 보험청구.고객ID = 고객.고객ID \nWHERE 보험청구.청구일 = (\n    SELE

In [24]:
df2 = pd.DataFrame(results_all2)
df2.head(2)


,Plan,Schema,Qeustion,Query,Reason,Result,difficulty,domain,sql_task_type,sql_complexity,sql_type,main_syntax
0,"보험 도메인에서 고객의 보험 청구 데이터를 관리하는 테이블을 설정하고, 특정 조건에...",DB: insurance_db\nTABLE DDL:\nCREATE TABLE `고객...,가장 최근에 보험 청구를 한 고객의 이름과 청구 금액을 알려줘.,"SELECT 고객.이름, 보험청구.청구금액 \nFROM 보험청구 \nINNER JO...",보험 청구 테이블에서 가장 최근 청구일을 찾기 위해 서브쿼리를 사용하여 MAX 함수...,True,4,보험,데이터 품질 관리 및 검증,subqueries,SELECT,"[OFFSET, INNER JOIN, WINDOW FUNCTION, HAVING, ..."
1,소셜 미디어 도메인에서 사용자의 특정 활동을 삭제하는 시나리오를 설정합니다. 4개의...,DB: social_media_db\nTABLE DDL:\nCREATE TABLE ...,지난 1년 동안 활동이 없는 사용자의 모든 활동 기록을 삭제해줘.,DELETE FROM 활동 WHERE 사용자ID IN (\n SELECT DI...,이 쿼리는 지난 1년 동안 활동이 없는 사용자의 활동 기록을 삭제하기 위해 작성되었...,False,4,소셜 미디어,데이터 수정 및 갱신,basic SQL,DELETE,"[DISTINCT, INTERSECT, HAVING, GROUP BY, COUNT,..."


In [25]:
df2.to_csv("results_all2.csv", index=False)

In [30]:
df_all = pd.concat([df1, df2])
df_all.head(2)

,Plan,Schema,Qeustion,Query,Reason,Result,difficulty,domain,sql_task_type,sql_complexity,sql_type,main_syntax
0,공공 보건 분야에서 백업 및 복구 관리를 위한 DELETE 쿼리를 작성합니다. 주어...,DB: public_health_db\nTABLE DDL:\nCREATE TABLE...,주민등록번호가 '123456-7890123'인 사람의 백신 접종 기록을 삭제해줘.,DELETE FROM 백신_접종_기록 WHERE 주민등록번호 IN ('123456-...,주어진 조건에 따라 주민등록번호가 특정 값인 사람의 백신 접종 기록을 삭제하는 쿼리...,True,1,공공 보건,백업 및 복구 관리,multiple joins,DELETE,"[WHERE, IN]"
1,공공 보건 분야에서 특정 조건에 맞는 데이터를 삭제하는 쿼리를 작성합니다. 주어진 ...,DB: public_health_db\nTABLE DDL:\nCREATE TABLE...,나이가 60세 이상인 환자들의 기록을 삭제해줘.,DELETE FROM 환자 WHERE 나이 IN (SELECT 나이 FROM 환자 ...,나이가 60세 이상인 환자들의 기록을 삭제하기 위해 DELETE 문을 사용합니다. ...,True,1,공공 보건,데이터 삭제,aggregation,UPDATE,[IN]


In [32]:
df_all["Result"].value_counts()

Result
True     1676
False      98
Name: count, dtype: int64

In [33]:
df_all_true = df_all[df_all["Result"] == True]
df_all_true.shape


(1676, 12)

In [37]:
from datasets import Dataset
def upload_to_huggingface(df_all_true, dataset_name="daje/synthetic-ko-sql-hard"):
    # 파일에서 데이터셋 로드
    # Hugging Face 데이터셋 생성
    hf_dataset = Dataset.from_pandas(df_all_true)
    
    # Hugging Face에 업로드
    hf_dataset.push_to_hub(dataset_name, token=os.getenv("SELF_HUGGINGFACE_TOKEN"))
    print(f"데이터셋을 '{dataset_name}'에 업로드했습니다.")

upload_to_huggingface(df_all_true)

Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.83s/it]


데이터셋을 'daje/synthetic-ko-sql-hard'에 업로드했습니다.


In [2]:
from datasets import load_dataset

dataset = load_dataset("daje/synthetic-ko-sql-hard", split="train")
dataset[0]


{'Plan': '공공 보건 분야에서 백업 및 복구 관리를 위한 DELETE 쿼리를 작성합니다. 주어진 조건에 따라 단일 테이블을 사용하고, WHERE와 IN 문법을 활용하여 특정 조건에 맞는 데이터를 삭제하는 쿼리를 생성합니다.',
 'Schema': 'DB: public_health_db\nTABLE DDL:\nCREATE TABLE `백신_접종_기록` (\n    접종ID INT NOT NULL,\n    주민등록번호 VARCHAR(13),\n    백신종류 VARCHAR(50),\n    접종일 DATE,\n    접종기관 VARCHAR(100),\n    PRIMARY KEY (접종ID)\n)',
 'Qeustion': "주민등록번호가 '123456-7890123'인 사람의 백신 접종 기록을 삭제해줘.",
 'Query': "DELETE FROM 백신_접종_기록 WHERE 주민등록번호 IN ('123456-7890123');",
 'Reason': "주어진 조건에 따라 주민등록번호가 특정 값인 사람의 백신 접종 기록을 삭제하는 쿼리를 작성했습니다. DELETE 문을 사용하여 특정 조건에 맞는 데이터를 삭제하며, WHERE 절과 IN 연산자를 사용하여 주민등록번호가 '123456-7890123'인 레코드를 선택했습니다. 이는 단일 테이블에서 특정 조건에 맞는 데이터를 삭제하는 간단한 작업입니다.",
 'Result': True,
 'difficulty': 1,
 'domain': '공공 보건',
 'sql_task_type': '백업 및 복구 관리',
 'sql_complexity': 'multiple joins',
 'sql_type': 'DELETE',
 'main_syntax': ['WHERE', 'IN'],
 '__index_level_0__': 0}

In [3]:
from collections import Counter
Counter(list(dataset["difficulty"]))

Counter({5: 712, 4: 344, 2: 216, 3: 207, 1: 197})

In [4]:
# requests 라이브러리를 가져와 HTTP 요청을 보낼 수 있게 함
import requests

def get_response(input_text):
    # vLLM API 엔드포인트 URL을 지정함 - 로컬 서버의 8100 포트로 설정되어 있음
    url = "http://localhost:8100/v1/chat/completions"

    # HTTP 요청 헤더를 설정함 - Content-Type을 application/json으로 지정하여 JSON 형식의 데이터를 전송함을 서버에 알림
    headers = {
        "Content-Type": "application/json",
    }

    # API 요청에 필요한 데이터를 JSON 형식으로 구성함
    data = {
        "model": "Qwen/Qwen2.5-Coder-7B-Instruct",  # 사용할 모델을 지정함 - Qwen의 코더 특화 모델임
        "messages": [                               # 대화 메시지 배열 구성
            {
                "role": "user",                     # 메시지 역할을 사용자로 지정함
                "content": f"{input_text}"          # 함수 인자로 받은 입력 텍스트를 메시지 내용으로 설정함
            }
        ]
    }

    # POST 요청을 보내고 응답을 받음 - headers와 data를 JSON 형식으로 전송함
    response = requests.post(url, headers=headers, json=data)

    # 응답 상태 코드를 확인하여 성공(200)인 경우와 실패한 경우를 분기 처리함
    if response.status_code == 200:
        # 응답이 성공적이면 JSON으로 파싱하여 결과를 추출함
        result = response.json()
        
        # 주석 처리된 코드: SQL 코드만 추출하는 부분 - 현재는 전체 응답을 반환하도록 변경됨
        # return result["choices"][0]["message"]["content"].split("```sql")[1].split("```")[0]
        
        # 모델의 응답 내용을 반환함 - choices 배열의 첫 번째 항목에서 메시지 내용을 가져옴
        return result["choices"][0]["message"]["content"]
    else:
        # 요청이 실패한 경우 오류 메시지를 생성하여 반환함 - 디버깅을 위한 정보 포함
        return f"Request failed: {response.status_code}, {response.text}"

In [5]:
# 배치 단위로 처리 (대량 데이터에 효율적)
def process_batch(examples):
    # 배치에 있는 모든 예제에 대해 새 컬럼 생성 - 배치 처리의 결과물을 담을 빈 리스트를 초기화함
    new_texts = []
    
    # 배치 내의 각 항목을 순회하며 처리함 - 배치는 여러 항목의 컬럼별 데이터를 담고 있는 딕셔너리 형태임
    for i in range(len(examples["Schema"])):
        # 번역된 지시사항이 있으면 이를 사용하고, 없으면 원본 지시사항을 사용함 - 데이터 처리 과정의 유연성을 제공함
        Schema = examples["Schema"][i]
        Qeustion = examples["Qeustion"][i]

        # 새로운 입력 텍스트를 구성함 - SQL 쿼리 생성을 위한 프롬프트 형식을 정의함
        new_text = f"""다음은 SQL 테이블 스키마와 수행할 작업에 대한 설명입니다. SQLite 문법에 맞는 쿼리만 작성해주세요. 설명이나 주석 없이 실행 가능한 SQL 쿼리 코드만 출력하세요.
### Instruction: {Qeustion}
### Input: {Schema}"""
        # 생성된 새 텍스트를 결과 리스트에 추가함 - 모든 항목에 대한 처리 결과를 모음
        new_texts.append(new_text)
    
    # 데이터셋에 새 컬럼 추가 - 원본 데이터를 유지하면서 처리된 결과를 새 컬럼으로 추가함
    examples["new_input"] = new_texts
    
    # 수정된 examples 딕셔너리를 반환함 - Hugging Face 데이터셋의 map 메서드는 이 반환값을 사용하여 데이터셋을 업데이트함
    return examples

# 배치 크기 지정하여 적용 - 데이터셋의 map 메서드로 배치 처리 함수를 적용함
# batched=True: 배치 모드 활성화하여 데이터를 배치 단위로 처리함
# batch_size=100: 한 번에 100개 항목을 처리하여 메모리 효율성과 처리 속도 간의 균형을 맞춤
updated_dataset = dataset.map(process_batch, batched=True, batch_size=100)

In [ ]:
print(updated_dataset["new_input"][0])

다음은 SQL 테이블 스키마와 수행할 작업에 대한 설명입니다. SQLite 문법에 맞는 쿼리만 작성해주세요. 설명이나 주석 없이 실행 가능한 SQL 쿼리 코드만 출력하세요.
### Instruction: 주민등록번호가 '123456-7890123'인 사람의 백신 접종 기록을 삭제해줘.
### Input: DB: public_health_db
TABLE DDL:
CREATE TABLE `백신_접종_기록` (
    접종ID INT NOT NULL,
    주민등록번호 VARCHAR(13),
    백신종류 VARCHAR(50),
    접종일 DATE,
    접종기관 VARCHAR(100),
    PRIMARY KEY (접종ID)
)


In [7]:
get_response(updated_dataset["new_input"][0])

"```sql\nDELETE FROM `백신_접종_기록`\nWHERE 주민등록번호 = '123456-7890123';\n```"

In [8]:
updated_dataset["Query"][0]

"DELETE FROM 백신_접종_기록 WHERE 주민등록번호 IN ('123456-7890123');"

In [9]:
from tqdm.auto import tqdm
before_inference = [get_response(temp['new_input']) for temp in tqdm(updated_dataset)]

100%|██████████| 1676/1676 [1:08:41<00:00,  2.46s/it]   


In [14]:
updated_dataset = dataset.add_column(name="llm_result", column=before_inference)
updated_dataset

Dataset({
    features: ['Plan', 'Schema', 'Qeustion', 'Query', 'Reason', 'Result', 'difficulty', 'domain', 'sql_task_type', 'sql_complexity', 'sql_type', 'main_syntax', '__index_level_0__', 'llm_result'],
    num_rows: 1676
})

In [37]:
updated_dataset.push_to_hub("daje/synthetic-ko-sql-hard-add-llm-result", token=os.getenv("SELF_HUGGINGFACE_TOKEN"))

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.48s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/daje/synthetic-ko-sql-hard-add-llm-result/commit/47a0db625a7b57b17f7384ce313e54d1b6fff386', commit_message='Upload dataset', commit_description='', oid='47a0db625a7b57b17f7384ce313e54d1b6fff386', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/daje/synthetic-ko-sql-hard-add-llm-result', endpoint='https://huggingface.co', repo_type='dataset', repo_id='daje/synthetic-ko-sql-hard-add-llm-result'), pr_revision=None, pr_num=None)

In [20]:
updated_dataset[0]

{'Plan': '공공 보건 분야에서 백업 및 복구 관리를 위한 DELETE 쿼리를 작성합니다. 주어진 조건에 따라 단일 테이블을 사용하고, WHERE와 IN 문법을 활용하여 특정 조건에 맞는 데이터를 삭제하는 쿼리를 생성합니다.',
 'Schema': 'DB: public_health_db\nTABLE DDL:\nCREATE TABLE `백신_접종_기록` (\n    접종ID INT NOT NULL,\n    주민등록번호 VARCHAR(13),\n    백신종류 VARCHAR(50),\n    접종일 DATE,\n    접종기관 VARCHAR(100),\n    PRIMARY KEY (접종ID)\n)',
 'Qeustion': "주민등록번호가 '123456-7890123'인 사람의 백신 접종 기록을 삭제해줘.",
 'Query': "DELETE FROM 백신_접종_기록 WHERE 주민등록번호 IN ('123456-7890123');",
 'Reason': "주어진 조건에 따라 주민등록번호가 특정 값인 사람의 백신 접종 기록을 삭제하는 쿼리를 작성했습니다. DELETE 문을 사용하여 특정 조건에 맞는 데이터를 삭제하며, WHERE 절과 IN 연산자를 사용하여 주민등록번호가 '123456-7890123'인 레코드를 선택했습니다. 이는 단일 테이블에서 특정 조건에 맞는 데이터를 삭제하는 간단한 작업입니다.",
 'Result': True,
 'difficulty': 1,
 'domain': '공공 보건',
 'sql_task_type': '백업 및 복구 관리',
 'sql_complexity': 'multiple joins',
 'sql_type': 'DELETE',
 'main_syntax': ['WHERE', 'IN'],
 '__index_level_0__': 0,
 'llm_result': "DELETE FROM `백신_접종_기록`\nWHERE 주민등록번호 = '123456-7890123';",
 'new_input': "다음은 SQL 테이블 스키마와 수행할

In [36]:
from google import genai
from google.genai import types
from pydantic import BaseModel 
import os
from pqdm.processes import pqdm
from functools import partial

class json_output(BaseModel):
    reason: str
    score: bool

client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

def evaluation_log(data):
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=f"""너는 주어진 SQL system prompt와 내가 만든 model이 SQL system prompt를 보고 만든 생성결과와 실제 정답을 너에게 전달해줄게. 이를 보고 모델이 알맞게 예측하였는지 평가하는 AI Asisstant입니다.
# SQL system prompt
{data['new_input']}
# model 생성결과
{data['llm_result']}
# 실제 정답
{data['Query']}

# 평가 기준
- reason은 한국어로 작성하세요.
-score는 모델이 의미적으로 같은 결과를 조회하는 쿼리라면 True, 아니면 False로 평가하세요.
""",
        config=types.GenerateContentConfig(
            max_output_tokens=1000,
            temperature=0.0,
            response_mime_type="application/json",
            response_schema=json_output,
        ),
    )
    model_response = response.candidates[0].content.parts[0].text
    return model_response

# 인덱스와 데이터셋을 받아 처리하는 함수
def process_item(idx, dataset):
    item = dataset[idx]
    return evaluation_log(item)

# 데이터셋의 인덱스만 전달 (처음 2개의 인덱스)
indices = list(range(len(updated_dataset)))  # [0, 1]

# partial 함수를 사용하여 dataset 매개변수를 고정
process_with_dataset = partial(process_item, dataset=updated_dataset)

# 병렬 처리 실행
result_all_ver1 = pqdm(indices, process_with_dataset, n_jobs=5)

QUEUEING TASKS | : 100%|██████████| 1676/1676 [00:00<00:00, 8712.39it/s]
PROCESSING TASKS | : 100%|██████████| 1676/1676 [07:55<00:00,  3.52it/s]
COLLECTING RESULTS | : 100%|██████████| 1676/1676 [00:00<00:00, 997538.46it/s]


In [39]:
score_list = [json_repair.loads(temp_dict)["score"] for temp_dict in result_all_ver1]

In [44]:
print(f"score 평균: {sum(score_list) / len(score_list):.2f}")

score 평균: 0.66
